# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#Dependencies
import requests
import random
import numpy as np
import pandas as pd
from citipy import citipy
# Import API key
from api_keys import weather_api_key



## Generate Cities List

In [2]:
#lat south - 90 eq 0 north 90 55 139   21
#long -180 to 180
cities = []
while len(cities) < 10:
    lat = round(np.random.uniform(-90.0, 90.0),2)
    lon = round(np.random.uniform(-180.0, 180.0),2)
    city = citipy.nearest_city(lat, lon).city_name
    cities.append(city)
    cities = list(set(cities))
print(cities)
len(list(set(cities)))

['bredasdorp', 'pevek', 'norman wells', 'hare bay', 'fortuna', 'qaanaaq', 'illoqqortoormiut', 'san quintin', 'ushuaia', 'punta arenas']


10

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:

print("Begining of data retrieval/n-----------------")
cities2 = []
lat = []
lon = []
maxtemp = []
humidity = []
cloudiness =[]
wind = []
country = []
date = []
i = 1
for city in cities:

   
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units=metric&appid={weather_api_key}"  
        data = requests.get(url).json()
        lat.append(data["coord"]["lat"])
        lon.append(data["coord"]["lat"])
        maxtemp.append(data["main"]["temp_max"])
        humidity.append(data["main"]["humidity"])
        cloudiness.append(data["clouds"]["all"])
        wind.append(data["wind"]["speed"])
        country.append(data["sys"]["country"])
        date.append(data["dt"])
    except: 
        message = data["message"]
        print(f"{message.capitalize()}. Skipping...")     
    else:
        print(f"Processing Record {i} of Set {len(cities)} | {city}")
        cities2.append(city)
        i +=1
        
citiesdata = {
    'City': cities2,
    'Lat': lat, 
    'Lng': lon, 
    "Max Temp" : maxtemp, 
    "Humidity" : humidity, 
    "Cloudiness" : cloudiness, 
    "Wind Speed": wind, 
    "Country": country,
    "Date" : date
}
# "cod":200

# {"cod":"404","message":"city not found"}

Begining of data retrieval/n-----------------
Processing Record 1 of Set 10 | bredasdorp
Processing Record 2 of Set 10 | pevek
Processing Record 3 of Set 10 | norman wells
Processing Record 4 of Set 10 | hare bay
Processing Record 5 of Set 10 | fortuna
Processing Record 6 of Set 10 | qaanaaq
City not found. Skipping...
Processing Record 7 of Set 10 | san quintin
Processing Record 8 of Set 10 | ushuaia
Processing Record 9 of Set 10 | punta arenas


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [4]:
print(cities)
print(citiesdata)

['bredasdorp', 'pevek', 'norman wells', 'hare bay', 'fortuna', 'qaanaaq', 'illoqqortoormiut', 'san quintin', 'ushuaia', 'punta arenas']
{'City': ['bredasdorp', 'pevek', 'norman wells', 'hare bay', 'fortuna', 'qaanaaq', 'san quintin', 'ushuaia', 'punta arenas'], 'Lat': [-34.5322, 69.7008, 65.282, 48.8499, 40.5982, 77.484, 30.4833, -54.8, -53.15], 'Lng': [-34.5322, 69.7008, 65.282, 48.8499, 40.5982, 77.484, 30.4833, -54.8, -53.15], 'Max Temp': [17, -15.74, 3, -2, 6.67, -23.88, 12.97, 6, 5], 'Humidity': [88, 84, 65, 93, 100, 58, 75, 52, 87], 'Cloudiness': [75, 9, 20, 11, 1, 20, 100, 0, 100], 'Wind Speed': [1.03, 2.7, 1.09, 1.03, 3.6, 3.63, 5.55, 3.09, 3.6], 'Country': ['ZA', 'RU', 'CA', 'CA', 'US', 'GL', 'MX', 'AR', 'CL'], 'Date': [1618470471, 1618470713, 1618470713, 1618470713, 1618470713, 1618470714, 1618470714, 1618470714, 1618470715]}


In [5]:
df1 = pd.DataFrame(citiesdata)
df1.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bredasdorp,-34.5322,-34.5322,17.00,88,75,1.03,ZA,1618470471
1,pevek,69.7008,69.7008,-15.74,84,9,2.70,RU,1618470713
2,norman wells,65.2820,65.2820,3.00,65,20,1.09,CA,1618470713
3,hare bay,48.8499,48.8499,-2.00,93,11,1.03,CA,1618470713
4,fortuna,40.5982,40.5982,6.67,100,1,3.60,US,1618470713


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [6]:
if len(df1[df1["Humidity"]>100]) == 0:
    print(0)
else:
    over100 = (df1[df1["Humidity"]>100]).index
    

0


In [8]:
#  Get the indices of cities that have humidity over 100%.
try:
    clean_city_data = df1.loc[over100]
except:
    clean_city_data = df1

In [9]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

clean_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bredasdorp,-34.5322,-34.5322,17.00,88,75,1.03,ZA,1618470471
1,pevek,69.7008,69.7008,-15.74,84,9,2.70,RU,1618470713
2,norman wells,65.2820,65.2820,3.00,65,20,1.09,CA,1618470713
3,hare bay,48.8499,48.8499,-2.00,93,11,1.03,CA,1618470713
4,fortuna,40.5982,40.5982,6.67,100,1,3.60,US,1618470713
5,qaanaaq,77.4840,77.4840,-23.88,58,20,3.63,GL,1618470714
6,san quintin,30.4833,30.4833,12.97,75,100,5.55,MX,1618470714
7,ushuaia,-54.8000,-54.8000,6.00,52,0,3.09,AR,1618470714
8,punta arenas,-53.1500,-53.1500,5.00,87,100,3.60,CL,1618470715


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression